In [1]:
from strategies import random_nodes, learn_target_parents
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
from config import DATA_FOLDER
from strategies.simulator import GenerationConfig, SimulationConfig, simulate
%matplotlib inline
sns.set()
np.random.seed(1729)

In [2]:
N_NODES = 6
DAG_FOLDER = 'small'
STRATEGIES = {
    'random': random_nodes.random_strategy,
    'learn-parents': learn_target_parents.create_learn_target_parents(N_NODES-1, 10000)
}

### Generate and save DAGs

In [3]:
G_CONFIG = GenerationConfig(
    n_nodes=N_NODES,
    edge_prob=.5,
    n_dags=100
)
gdags = G_CONFIG.save_dags(DAG_FOLDER)

=== Saving DAGs ===
=== Saved ===


### Run simulation on each DAG

In [4]:
STRATEGY = 'random'
SIM_CONFIG = SimulationConfig(
    starting_samples=250,
    n_samples=100,
    n_batches=2,
    max_interventions=2,
    strategy=STRATEGY,
    intervention_strength=2,
)

for i, gdag in enumerate(gdags):
    print('=== Simulating strategy for DAG %d: %s' % (i, gdag))
    simulate(STRATEGIES[STRATEGY], SIM_CONFIG, gdag, os.path.join(DATA_FOLDER, DAG_FOLDER, 'dag%d' % i))

=== Simulating strategy for DAG 0: [1][2|1][3|2][0][5|0,1,2][4|0,3]
Batch 0
Batch 1
=== Simulating strategy for DAG 1: [0][5|0][4|0][2|0][1|0][3|0,1]
Batch 0
Batch 1
=== Simulating strategy for DAG 2: [0][3|0][1|0][4|1,3][5|0,4][2|1]
Batch 0
Batch 1
=== Simulating strategy for DAG 3: [1][4|1][0][3|0,1][5|0,1,3,4][2|0,1]
Batch 0
Batch 1
=== Simulating strategy for DAG 4: [1][0][2|0,1][5|1,2][4|0,2][3|0,1,2]
Batch 0
Batch 1
=== Simulating strategy for DAG 5: [0][1|0][4|0,1][5|0,1,4][2|1][3|2]
Batch 0
Batch 1
=== Simulating strategy for DAG 6: [1][0][3|0][4|3][2|0][5|0,2]
Batch 0
Batch 1
=== Simulating strategy for DAG 7: [2][0][1|0][5|0,1][3|0,1,2][4|2,3]
Batch 0
Batch 1
=== Simulating strategy for DAG 8: [1][0][2|0][3|1,2][4|1,2,3][5|2,4]
Batch 0
Batch 1
=== Simulating strategy for DAG 9: [2][0][1|0][3|1,2][5|0,3][4|0,1,2,3]
Batch 0
Batch 1
=== Simulating strategy for DAG 10: [1][2|1][0][3|0,1,2][4|0,1,3][5|1,2,4]
Batch 0
Batch 1
=== Simulating strategy for DAG 11: [2][0][3|0][1|0][5|0,

=== Simulating strategy for DAG 99: [2][1][5|1][0][3|0,1][4|1,3]
Batch 0
Batch 1


In [5]:
STRATEGY = 'learn-parents'
SIM_CONFIG = SimulationConfig(
    starting_samples=250,
    n_samples=100,
    n_batches=2,
    max_interventions=2,
    strategy=STRATEGY,
    intervention_strength=2,
)

for i, gdag in enumerate(gdags):
    print('=== Simulating strategy for DAG %d: %s' % (i, gdag))
    simulate(STRATEGIES[STRATEGY], SIM_CONFIG, gdag, os.path.join(DATA_FOLDER, DAG_FOLDER, 'dag%d' % i))

=== Simulating strategy for DAG 0: [1][2|1][3|2][0][5|0,1,2][4|0,3]
Batch 0
All files deleted in /Users/chandlersquires/Documents/active_learning/new/data/TEMP_DAGS/
Batch 1
All files deleted in /Users/chandlersquires/Documents/active_learning/new/data/TEMP_DAGS/
=== Simulating strategy for DAG 1: [0][5|0][4|0][2|0][1|0][3|0,1]
Batch 0
All files deleted in /Users/chandlersquires/Documents/active_learning/new/data/TEMP_DAGS/
Batch 1
All files deleted in /Users/chandlersquires/Documents/active_learning/new/data/TEMP_DAGS/
=== Simulating strategy for DAG 2: [0][3|0][1|0][4|1,3][5|0,4][2|1]
Batch 0
All files deleted in /Users/chandlersquires/Documents/active_learning/new/data/TEMP_DAGS/
Batch 1
All files deleted in /Users/chandlersquires/Documents/active_learning/new/data/TEMP_DAGS/
=== Simulating strategy for DAG 3: [1][4|1][0][3|0,1][5|0,1,3,4][2|0,1]
Batch 0
All files deleted in /Users/chandlersquires/Documents/active_learning/new/data/TEMP_DAGS/
Batch 1
All files deleted in /Users/chan

=== Simulating strategy for DAG 31: [3][0][4|0][1|0][5|0,1,4][2|1]
Batch 0
All files deleted in /Users/chandlersquires/Documents/active_learning/new/data/TEMP_DAGS/
Batch 1
All files deleted in /Users/chandlersquires/Documents/active_learning/new/data/TEMP_DAGS/
=== Simulating strategy for DAG 32: [0][1|0][3|1][2|0,1][4|1,2][5|1,4]
Batch 0
All files deleted in /Users/chandlersquires/Documents/active_learning/new/data/TEMP_DAGS/
Batch 1
All files deleted in /Users/chandlersquires/Documents/active_learning/new/data/TEMP_DAGS/
=== Simulating strategy for DAG 33: [2][1][5|1,2][0][4|0][3|0]
Batch 0
All files deleted in /Users/chandlersquires/Documents/active_learning/new/data/TEMP_DAGS/
Batch 1
All files deleted in /Users/chandlersquires/Documents/active_learning/new/data/TEMP_DAGS/
=== Simulating strategy for DAG 34: [1][2|1][4|1,2][3|2][0][5|0,1,2,3]
Batch 0
All files deleted in /Users/chandlersquires/Documents/active_learning/new/data/TEMP_DAGS/
Batch 1
All files deleted in /Users/chandl

KeyError: 5